# Multi terrain probabistic sensing

# About this file

This file is to use the functionalities of the classes defined in the other file.

## Things to be done here

1) write algorithm for agent3, agent4 <br>
2) if object_matrix is knowledge base, then lets rename it to knowledge base to avoid confusion and have unambiguous variable names.<br>
3) Import A* code into this file.

## Things done here

1) declared the set of global variables required
2) renamed object_matrix to knowledge_base

In [1]:
%run a_star.ipynb
import numpy as np
%run agent_6.ipynb

import random

In [2]:
maze_size = 10
probability = 0.3

# Code for Agent 6 goes here

In [3]:
# def agent6(maze_size, source): # using this name for the lack of a better name. please change.
#     global agent_gridworld
#     global full_grid_world_matrix
#     global terrain_matrix

#     _,_,_,path = astar(source,agent_gridworld,goal)
#     path = path[::-1]
#     print('path', path)
#     goal_reached = False

#     if path == []:
#         return visited_list, number_of_bumps, infered_list, goal_reached

#     for node in path:
#         row = node[0]
#         column = node[1]
#         knowledge_base[row][column].blocked = helper_matrix[row][col]

In [4]:
def run_agent_6(agent6, source):
    global terrain_matrix   
    global full_grid_world_matrix
    global agent_gridworld

    target_reached = False
    while not target_reached:
        current_target = agent6.getMaxProbabilityCell(source)
        print(current_target)
        _,_,_,path = astar(source,current_target,  agent6.agent_gridworld)
        print(path)
        for node in path:
            terrain_type = check_type(node)
            update_agent6_terrain_matrix()
            if terrain_type == 3:
                agent6_terrain_matrix[node[0],node[1]].belief = 0
                update_probabilites_when_blocked_formula()
                break    
        target_reached = examine(path[-1])

        if (not target_reached):
            update_probabilites_when_unblocked_formula()


# Main function

In [5]:
if __name__ == "__main__":

#     source,target=set_valid_source_target()
    maze = gridworld(maze_size, probability)
    agent6 = agent_6(maze_size)
    print(maze.source,maze.target)
    print(maze.full_grid_world_matrix)
    run_agent_6(agent6, maze.source)

source:target
0
0
source (5, 8)
target (0, 6)
(5, 8) (0, 6)
[[0 0 0 1 1 0 0 1 0 0]
 [0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [1 1 1 1 0 1 0 0 0 0]
 [0 1 0 0 1 1 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 1 1 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0]]
(4, 8)
(5, 8) (4, 8)
Reached Goal!!!!!!!!!!!!!!!!!!!!!!!
[(4, 8), (3, 8), (2, 8), (1, 8), (0, 8), (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9), (9, 8), (8, 8), (7, 8), (6, 8), (5, 8)]
[(4, 8), (3, 8), (2, 8), (1, 8), (0, 8), (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9), (9, 8), (8, 8), (7, 8), (6, 8), (5, 8)]


NameError: name 'check_type' is not defined

In [ ]:
plt.spy(maze.full_grid_world_matrix)